In [ ]:
# Import Libraries
import os
import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm
import matplotlib.pyplot as plt
from skimage.feature import hog, local_binary_pattern
from skimage.util import view_as_windows

In [ ]:
# Access the dataset folder
AllDatasetDirPath = r"D:\AI-Practical-Tasks\2023-2024\final\Group D\MangoLeafBD Dataset"
FolderList = my_list = os.listdir(AllDatasetDirPath)
print(FolderList)

In [ ]:
def create_dataset(dataset_dir):
    names = [f"haralick_{i+1}" for i in range(13)]
    names.append("classlabel")
    all_dataset_results = pd.DataFrame(columns=names)

    for _, dirs, _ in os.walk(dataset_dir):
        if len(dirs) > 0:
            categories = dirs
            break

    for category in categories:
        category_path = os.path.join(dataset_dir, category)
        class_label = categories.index(category)
        for filename in tqdm(os.listdir(category_path), desc=f"Processing {category}"):
            imgpath = os.path.join(category_path, filename)
            main_img = cv2.imread(imgpath)

            # Convert the image to Grayscale
            gray = cv2.cvtColor(main_img, cv2.COLOR_BGR2GRAY)

            # Texture based features
            textures = mt.features.haralick(gray)
            ht_mean = list(textures.mean(axis=0))
            ht_mean.append(class_label)

            # Append the feature vector to the DataFrame
            df_temp = pd.DataFrame([ht_mean], columns=names)
            all_dataset_results = pd.concat(
                [all_dataset_results, df_temp], ignore_index=True
            )

    return all_dataset_results

In [ ]:
dataset = create_dataset(AllDatasetDirPath)

In [ ]:
dataset.shape

In [ ]:
dataset.head()

In [ ]:
dataset.to_csv("haralick_features.csv")

In [ ]:
def overlay_features(image_path):
    # Read the image
    main_img = cv2.imread(image_path)
    original_img = main_img.copy()  # Create a copy for visualization

    # Preprocessing
    gray = cv2.cvtColor(main_img, cv2.COLOR_BGR2GRAY)
    _, thresholded = cv2.threshold(
        gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU
    )
    contours, _ = cv2.findContours(
        thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )
    leaf_contour = max(contours, key=cv2.contourArea)

    # Extract features
    x, y, w, h = cv2.boundingRect(leaf_contour)
    area = cv2.contourArea(leaf_contour)
    perimeter = cv2.arcLength(leaf_contour, True)
    physiological_length = max(w, h)
    physiological_width = min(w, h)
    aspect_ratio = float(physiological_length) / physiological_width

    # Visualize the image with features overlay
    cv2.rectangle(
        original_img, (x, y), (x + w, y + h), (0, 255, 0), 2
    )  # Draw bounding box

    # Annotate extracted features
    text = (
        f"Area: {area:.2f}, Perimeter: {perimeter:.2f},Aspect Ratio: {aspect_ratio:.2f}"
    )
    cv2.putText(
        original_img,
        text,
        (0, y + 300),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.266,
        (0, 0, 0),
        1,
        cv2.LINE_AA,
    )

    # Display the image with annotations
    plt.figure(figsize=(10, 18))
    plt.imshow(cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB))
    plt.title("Image with Extracted Features Overlay")
    plt.axis("off")
    plt.show()

In [ ]:
# Example usage
image_path = r"D:\AI-Practical-Tasks\2023-2024\final\Group D\MangoLeafBD Dataset\Anthracnose\20211008_124249 (Custom).jpg"  # Replace this with the path to your image file
overlay_features(image_path)

In [ ]:
# Read the image
img_path = r"D:\AI-Practical-Tasks\2023-2024\final\Group D\MangoLeafBD Dataset\Anthracnose\20211008_124249 (Custom).jpg"  # Replace with the actual path of your image
main_img = cv2.imread(img_path)

# Convert the image to Grayscale
gray = cv2.cvtColor(main_img, cv2.COLOR_BGR2GRAY)

# Extract SIFT features
sift = cv2.SIFT_create()
keypoints, descriptors = sift.detectAndCompute(gray, None)
sift_features = (
    descriptors.flatten() if descriptors is not None else np.zeros(128)
)  # Assuming 128-dimensional SIFT descriptors

# Compute HOG features
hogg = cv2.HOGDescriptor()
hog_features = hogg.compute(gray).flatten()

# Extract LBP features
lbp_radius = 3
lbp_points = 8 * lbp_radius
lbp = local_binary_pattern(gray, lbp_points, lbp_radius, method="uniform")
lbp_hist, _ = np.histogram(
    lbp.ravel(), bins=np.arange(0, lbp_points + 3), range=(0, lbp_points + 2)
)
lbp_features = lbp_hist / np.sum(lbp_hist)

# Extract Haralick texture features
textures = mh.features.haralick(gray)
ht_mean = textures.mean(axis=0)
haralick_features = ht_mean[:14]  # Assuming 14 Haralick features

# Compute Canny edges
edges = cv2.Canny(gray, 100, 200)

# Display the extracted features
print(
    "SIFT Features:", sift_features[:2]
)  # Displaying first 2 SIFT features as an example
print(
    "HOG Features:", hog_features[:2]
)  # Displaying first 2 HOG features as an example
print(
    "LBP Features:", lbp_features[:2]
)  # Displaying first 2 LBP features as an example
print(
    "Haralick Features:", haralick_features[:2]
)  # Displaying first 2 Haralick features as an example

# Visualizing LBP Features
plt.subplot(1, 2, 1)
lbp_img = lbp.astype(np.uint8)  # Convert LBP features to uint8 for visualization
plt.imshow(lbp_img, cmap="gray")
plt.title("Local Binary Patterns (LBP)")
plt.axis("off")

# Visualizing Canny Edges
plt.subplot(1, 2, 2)
plt.imshow(edges, cmap="gray")
plt.title("Canny Edges")
plt.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
def calculate_features(gray):
    # Extract SIFT features
    gray = cv2.resize(gray, (100, 100))
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    sift_features = descriptors.flatten() if descriptors is not None else np.zeros(128)

    return sift_features


def create_dataset(dataset_dir):
    names = ["SIFT"]

    all_dataset_results = pd.DataFrame(columns=names)

    categories = next(os.walk(dataset_dir))[1]

    for category in categories:
        category_path = os.path.join(dataset_dir, category)
        class_label = categories.index(category)
        for filename in tqdm(os.listdir(category_path), desc=f"Processing {category}"):
            imgpath = os.path.join(category_path, filename)
            main_img = cv2.imread(imgpath)

            # Convert the image to Grayscale
            gray = cv2.cvtColor(main_img, cv2.COLOR_BGR2GRAY)

            # Calculate features
            features = calculate_features(gray)

            # Append the feature vector to the DataFrame
            df_temp = pd.DataFrame([[features]], columns=names)
            df_temp["classlabel"] = class_label
            all_dataset_results = pd.concat(
                [all_dataset_results, df_temp], ignore_index=True
            )

    return all_dataset_results

In [ ]:
dataset = create_dataset(AllDatasetDirPath)

In [ ]:
dataset.to_csv("features1.csv")

In [ ]:
dataset.shape

In [ ]:
dataset.head()

In [ ]:
# Load an example image
image = io.imread(
    r"D:\AI-Practical-Tasks\2023-2024\final\Group D\MangoLeafBD Dataset\Anthracnose\20211008_124249 (Custom).jpg",
    as_gray=True,
)

# HOG feature extraction
hog_features, hog_image = hog(
    image,
    orientations=9,
    pixels_per_cell=(8, 8),
    cells_per_block=(3, 3),
    visualize=True,
    block_norm="L2-Hys",
)

# LBP feature extraction
radius = 3
n_points = 8 * radius
lbp = local_binary_pattern(image, n_points, radius, method="uniform")
print(len(hog_features), lbp.shape)
image_int = (image * 255).astype(int)  # Convert to integer type (0-255 range)

# Haralick feature extraction
textures = view_as_windows(image_int, (32, 32))
haralick_features = mh.features.haralick(textures[:, :, 0, 0]).mean(axis=0)
print(haralick_features)
# Displaying the results or further processing as needed
plt.figure(figsize=(12, 12))

plt.subplot(2, 3, 1)
plt.imshow(image, cmap="gray")
plt.title("Original Image")

plt.subplot(2, 3, 2)
plt.imshow(hog_image, cmap="gray")
plt.title("HOG Features")

plt.subplot(2, 3, 3)
plt.imshow(lbp, cmap="gray")
plt.title("LBP Features")

plt.subplot(2, 3, 4)
plt.bar(range(len(haralick_features)), haralick_features)
plt.title("Haralick Features")

plt.tight_layout()
plt.show()

In [ ]:
def create_dataset(dataset_dir):
    names = [f"hog_{i+1}" for i in range(7200)]
    names.append("classlabel")
    all_dataset_results = pd.DataFrame(columns=names)

    for _, dirs, _ in os.walk(dataset_dir):
        if len(dirs) > 0:
            categories = dirs
            break

    for category in categories:
        category_path = os.path.join(dataset_dir, category)
        class_label = categories.index(category)
        for filename in tqdm(os.listdir(category_path), desc=f"Processing {category}"):
            imgpath = os.path.join(category_path, filename)
            main_img = cv2.imread(imgpath)
            main_img = cv2.resize(main_img, (100, 100))
            gray = cv2.cvtColor(main_img, cv2.COLOR_BGR2GRAY)
            hog_features = hog(
                gray,
                orientations=8,
                pixels_per_cell=(8, 8),
                cells_per_block=(3, 3),
                visualize=False,
                block_norm="L2-Hys",
            )
            hog_features = list(hog_features)
            hog_features.append(class_label)
            # Append the feature vector to the DataFrame
            df_temp = pd.DataFrame([hog_features], columns=names)
            all_dataset_results = pd.concat(
                [all_dataset_results, df_temp], ignore_index=True
            )

    return all_dataset_results

In [ ]:
dataset = create_dataset(AllDatasetDirPath)

In [ ]:
dataset.shape

In [ ]:
dataset.head()

In [ ]:
dataset.to_csv("hog_features.csv")

In [ ]:
def create_dataset(dataset_dir):
    names = [f"lbp_{i+1}" for i in range(10000)]
    names.append("classlabel")
    all_dataset_results = pd.DataFrame(columns=names)

    for _, dirs, _ in os.walk(dataset_dir):
        if len(dirs) > 0:
            categories = dirs
            break

    for category in categories:
        category_path = os.path.join(dataset_dir, category)
        class_label = categories.index(category)
        for filename in tqdm(os.listdir(category_path), desc=f"Processing {category}"):
            imgpath = os.path.join(category_path, filename)
            main_img = cv2.imread(imgpath)
            main_img = cv2.resize(main_img, (100, 100))
            gray = cv2.cvtColor(main_img, cv2.COLOR_BGR2GRAY)
            radius = 3
            n_points = 8 * radius
            lbp = local_binary_pattern(gray, n_points, radius, method="uniform")
            # Append the feature vector to the DataFrame
            lbp = list(lbp.flatten())
            lbp.append(class_label)
            df_temp = pd.DataFrame([lbp], columns=names)
            all_dataset_results = pd.concat(
                [all_dataset_results, df_temp], ignore_index=True
            )

    return all_dataset_results

In [ ]:
dataset = create_dataset(AllDatasetDirPath)

In [ ]:
dataset.shape

In [ ]:
dataset.head()

In [ ]:
dataset.to_csv("lbp_features.csv")

In [ ]:
def create_dataset(dataset_dir):
    names = [f"edge_{i+1}" for i in range(10000)]
    names.append("classlabel")
    all_dataset_results = pd.DataFrame(columns=names)

    for _, dirs, _ in os.walk(dataset_dir):
        if len(dirs) > 0:
            categories = dirs
            break

    for category in categories:
        category_path = os.path.join(dataset_dir, category)
        class_label = categories.index(category)
        for filename in tqdm(os.listdir(category_path), desc=f"Processing {category}"):
            imgpath = os.path.join(category_path, filename)
            main_img = cv2.imread(imgpath)
            main_img = cv2.resize(main_img, (100, 100))
            gray = cv2.cvtColor(main_img, cv2.COLOR_BGR2GRAY)
            edges = cv2.Canny(gray, 100, 200)
            edges = list(edges.flatten())
            edges.append(class_label)
            df_temp = pd.DataFrame([edges], columns=names)
            all_dataset_results = pd.concat(
                [all_dataset_results, df_temp], ignore_index=True
            )

    return all_dataset_results

In [ ]:
dataset = create_dataset(AllDatasetDirPath)

In [ ]:
dataset.shape

In [ ]:
dataset.head()

In [ ]:
dataset.to_csv("features/edges.csv")